In [ ]:
### This tutorial will walk you through defining variables for controlling your OT Pipette

In [4]:
### Find V range ###

# In your Duet config file, find your steps/mm setting. This sould be either 48 or 200, depending on the generation
# of your pipette. The command in the confing will look like "M92 V48" or "M92 V200"

steps_mm = 48 # Enter steps/mm here
v_range = 9600/steps_mm
max_vol = 50 # Enter the max volume of your pipette in mircoliters
aspiration_home = v_range*0.76 # 76% of the motor range is used for liquid handling. the rest is for ejecting tips
steps_microliter = aspiration_home/max_vol

In [ ]:
### Open your MachineUtils.py in jupyter notebooks ###

In [12]:
### Add v axis movement to "moveTo" function ###

# You can copy and replace your "moveTo function with the print output from this cell
# If your machine has additional axes, you can instead add v-axis parts where appropriate
 

print("def moveTo(self, x=None, y=None, z=None, v=None, s = 6000):") # adds 'v=None' to parameters
print()
print("    if v and (v > " + str(v_range) + " or v < 0):") # checks to make sure the motor will stay in range
print("        v=None")
print("        raise Exception ('V cannot be less than O or greater than " + str(v_range) + "')")
print()
print("    x = \"{0:.2f}\".format(x) if x is not None else None")
print("    y = \"{0:.2f}\".format(y) if y is not None else None")
print("    z = \"{0:.2f}\".format(z) if z is not None else None")
print("    v = \"{0:.2f}\".format(v) if v is not None else None") # adds formatting for duet control
print("    s = \"{0:.2f}\".format(s)")
print()
print("    x_cmd = y_cmd = z_cmd = v_cmd = f_cmd \'\'") # initializes command variables
print()
print("    if x is not None:")
print("        x_cmd = f\'X{x}\'")
print("    if y is not None:")
print("        y_cmd = f\'Y{y}\'")
print("    if z is not None:")
print("        z_cmd = f\'Z{z}\'")
print("    if v is not None:")
print("        v_cmd = f\'V{v}\'") # assigns command variable
print("    if s is not None:")
print("        f_cmd = f\'F{s}\'")
print()
print("    self.setAbsolute()")
print("    cmd = f\"G0 {x_cmd} {y_cmd} {z_cmd} {v_cmd} {f_cmd}\"")
print("    self.send(cmd)")

def moveTo(self, x=None, y=None, z=None, v=None, s = 6000):

    if v and (v > 200.0 or v < 0):
        v=None
        raise Exception ('V cannot be less than O or greater than 200.0')

    x = "{0:.2f}".format(x) if x is not None else None
    y = "{0:.2f}".format(y) if y is not None else None
    z = "{0:.2f}".format(z) if z is not None else None
    v = "{0:.2f}".format(v) if v is not None else None
    s = "{0:.2f}".format(s)

    x_cmd = y_cmd = z_cmd = v_cmd = f_cmd ''

    if x is not None:
        x_cmd = f'X{x}'
    if y is not None:
        y_cmd = f'Y{y}'
    if z is not None:
        z_cmd = f'Z{z}'
    if v is not None:
        v_cmd = f'V{v}'
    if s is not None:
        f_cmd = f'F{s}'

    self.setAbsolute()
    cmd = f"G0 {x_cmd} {y_cmd} {z_cmd} {v_cmd} {f_cmd}"
    self.send(cmd)


In [16]:
### Add function to prime the pipette for aspiration ###

# Copy the output of this cell somewhere below the "moveTo" function

print("    def aspirate_prime(self):")
print("        self.moveTo(v="+ str(aspiration_home) +")")

    def aspirate_prime(self):
        self.moveTo(v=152.0)


In [20]:
### Add function to eject tip ###

# Copy the output of this cell somewhere below the "moveTo" function

print("    def eject_tip(self):")
print("        self.moveTo(v="+ str(v_range*0.95) +")") # pushes off tip
print("        self.aspirate_prime()") # resets position to aspirate

    def eject_tip(self):
        self.moveTo(v=190.0)
        self.aspirate_prime()


In [11]:
### Add function to aspirate liquid ###

# Copy the output of this cell somewhere below the "moveTo" and  "getPosition" function

print("    def aspirate(self, vol): # volume is in microliters")
print("        dv=-(vol*"+ str(steps_microliter)+")") # this divides motor range by max liquid in microliters to
                                                         # get a rough calculation of steps/microliter
    
print("        end_pos = float(self.getPosition()['V']) + dv") # this calculates the final v position by retrieving
                                                               # the current position and add the change in position
print("        self.moveTo(v=end_pos)")

    def aspirate(self, vol): # volume is in microliters
        dv=-(vol*3.04)
        end_pos = float(self.getPosition()['V']) + dv
        self.moveTo(v=end_pos)


In [8]:
### Add function to dispense liquid ###

# Copy the output of this cell somewhere below the "moveTo" and  "getPosition" function

print("    def dispense(self, vol): # volume is in microliters. 50 is the max volume")
print("        dv=(vol*"+ str(steps_microliter)+")")
print("        end_pos = float(self.getPosition()['V']) + dv")
print("        self.moveTo(v=end_pos)")

    def dispense(self, vol): # volume is in microliters. 50 is the max volume
        dv=(vol*3.04)
        end_pos = float(self.getPosition()['V']) + dv
        self.moveTo(v=end_pos)


In [ ]:
### The OT pipette should now be safe to use and roughly calibrated ###